# My First Query

One of the most powerful features of Marvin 2.0 is ability to query the newly created DRP and DAP databases. You can do this in two ways:
1. via the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)** or
2. via Python (in the terminal/notebook/script) with **Marvin-tools**.

The best part is that both interfaces use the same underlying query structure, so your input search will be the same. Here we will run a few queries with Marvin-tools to learn the basics of how to construct a query and also test drive some of the more advanced features that are unique to the Marvin-tools version of querying.

In [1]:
# Python 2/3 compatibility
from __future__ import print_function, division, absolute_import

In [2]:
from marvin import config
config.setRelease('MPL-4')

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named attr'.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


INFO: No release version set. Setting default to MPL-6


In [3]:
from marvin.tools.query import Query

Let's search for galaxies with M$_\star$ > 3 $\times$ 10$^{11}$ M$_\odot$.

To specify our search parameter, M$_\star$, we must know the database table and name of the parameter. In this case, MaNGA uses the NASA-Sloan Atlas (NSA) for target selection so we will use the Sersic profile determination for stellar mass, which is the **`sersic_mass`** parameter of the **`nsa`** table, so our search parameter will be **`nsa.sersic_mass`**.  You can also use **`nsa.sersic_logmass`**

Generically, the search parameter will take the form **`table.parameter`**.

In [4]:
myquery1 = 'nsa.sersic_mass > 3e11'
# or
myquery1 = 'nsa.sersic_logmass > 11.47'

In [5]:
q1 = Query(search_filter=myquery1)
r1 = q1.run()

Your parsed filter is: 
nsa.sersic_logmass>11.47


Running the query produces a **`Results`** object (**`r1`**):

In [6]:
# show results
r1.results

<ResultSet(set=1/1, index=0:9, count_in_set=9, total=9)>
[ResultRow(mangaid=u'1-134760', plate=8555, plateifu=u'8555-12701', ifu_name=u'12701', sersic_logmass=11.7491099236116),
 ResultRow(mangaid=u'1-377841', plate=8133, plateifu=u'8133-12705', ifu_name=u'12705', sersic_logmass=11.5011565223301),
 ResultRow(mangaid=u'1-278075', plate=8257, plateifu=u'8257-6104', ifu_name=u'6104', sersic_logmass=11.5671758130693),
 ResultRow(mangaid=u'1-256301', plate=8258, plateifu=u'8258-9102', ifu_name=u'9102', sersic_logmass=12.8522847612299),
 ResultRow(mangaid=u'1-276590', plate=8254, plateifu=u'8254-6103', ifu_name=u'6103', sersic_logmass=11.4709056630131),
 ResultRow(mangaid=u'1-217041', plate=8247, plateifu=u'8247-9102', ifu_name=u'9102', sersic_logmass=11.5166583074443),
 ResultRow(mangaid=u'1-338090', plate=8131, plateifu=u'8131-3701', ifu_name=u'3701', sersic_logmass=12.060916820615),
 ResultRow(mangaid=u'1-377378', plate=8132, plateifu=u'8132-6102', ifu_name=u'6102', sersic_logmass=11.7106

We will learn how to use the features of our **`Results`** object a little bit later, but first let's revise our search to see how more complex search queries work.

## Multiple Search Criteria

Let's add to our search to find only galaxies with a redshift less than 0.1.

Redshift is the **`z`** parameter and is also in the **`nsa`** table, so its full search parameter designation is **`nsa.z`**.

In [7]:
myquery2 = 'nsa.sersic_mass > 3e11 AND nsa.z < 0.1'

In [8]:
q2 = Query(search_filter=myquery2)
r2 = q2.run()

Your parsed filter is: 
and_(nsa.sersic_mass>3e11, nsa.z<0.1)


In [9]:
r2.results

<ResultSet(set=1/1, index=0:4, count_in_set=4, total=4)>
[ResultRow(mangaid=u'1-256301', plate=8258, plateifu=u'8258-9102', ifu_name=u'9102', z=0.0961407, sersic_mass=7116800000000.0),
 ResultRow(mangaid=u'1-338090', plate=8131, plateifu=u'8131-3701', ifu_name=u'3701', z=0.0806724, sersic_mass=1150580000000.0),
 ResultRow(mangaid=u'1-377378', plate=8132, plateifu=u'8132-6102', ifu_name=u'6102', z=0.0265521, sersic_mass=513575000000.0),
 ResultRow(mangaid=u'1-283711', plate=8317, plateifu=u'8317-1901', ifu_name=u'1901', z=0.094444, sersic_mass=2579810000000.0)]

## Compound Search Statements

We were hoping for a few more than 3 galaxies, so let's try to increase our search by broadening the criteria to also include galaxies with 127 fiber IFUs and a b/a ratio of at least 0.95.

To find 127 fiber IFUs, we'll use the **`name`** parameter of the **`ifu`** table, which means the full search parameter is **`ifu.name`**. However, **`ifu.name`** returns the IFU design name, such as `12701`, so we need to to set the value to `127*`.

The b/a ratio is in **`nsa`** table as the **`elpetro_ba`** parameter.

We're also going to join this to or previous query with an **`OR`** operator and use parentheses to group our individual search statements into a compound search statement.

In [13]:
myquery3 = '(nsa.sersic_mass > 3e11 AND nsa.z < 0.1) OR (ifu.name=127* AND nsa.elpetro_ba >= 0.95)'

In [14]:
q3 = Query(search_filter=myquery3)
r3 = q3.run()

Your parsed filter is: 
or_(and_(nsa.sersic_mass>3e11, nsa.z<0.1), and_(ifu.name=127*, nsa.elpetro_ba>=0.95))


In [15]:
r3.results

<ResultSet(set=1/1, index=0:16, count_in_set=16, total=16)>
[ResultRow(mangaid=u'1-90849', plate=8481, plateifu=u'8481-12703', ifu_name=u'12703', elpetro_ba=0.973022, z=0.0661473, sersic_mass=49861900000.0),
 ResultRow(mangaid=u'1-633196', plate=8485, plateifu=u'8485-12703', ifu_name=u'12703', elpetro_ba=0.95488, z=0.0380864, sersic_mass=84212200000.0),
 ResultRow(mangaid=u'1-251764', plate=8335, plateifu=u'8335-12702', ifu_name=u'12702', elpetro_ba=0.956245, z=0.0188389, sersic_mass=1883260000.0),
 ResultRow(mangaid=u'1-285095', plate=8319, plateifu=u'8319-12701', ifu_name=u'12701', elpetro_ba=0.961789, z=0.0571118, sersic_mass=132829000000.0),
 ResultRow(mangaid=u'1-258425', plate=8261, plateifu=u'8261-12702', ifu_name=u'12702', elpetro_ba=0.989192, z=0.112247, sersic_mass=84189200000.0),
 ResultRow(mangaid=u'1-256301', plate=8258, plateifu=u'8258-9102', ifu_name=u'9102', elpetro_ba=0.969656, z=0.0961407, sersic_mass=7116800000000.0),
 ResultRow(mangaid=u'1-338090', plate=8131, plate

## Design Your Own Search

OK, now it's your turn to try designing a search.

**Exercise**: Write a search filter that will find galaxies with a redshift less than 0.02 that were observed with the 1901 IFU?

In [16]:
# Enter your search here

## Finding the Available Parameters
Now you might want to go out and try all of the interesting queries that you've been saving up, but you don't know what the parameters are called or what database table they are in. 

You can find all of the availabale parameters by:
1. clicking on in the **Return Parameters** dropdown menu on the left side of the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)**,
2. reading the **[Marvin Docs page](https://api.sdss.org/doc/manga/marvin/refactor/query_params.html#marvin-query-parameters)**, or
3. via **Marvin-tools** (see next two cells)

In [17]:
# You might have to do an svn update to get this to work (otherwise try the next cell)
q = Query()
q.get_available_params()

[<ParameterGroup name=Metadata, n_parameters=7>,
 <ParameterGroup name=Spaxel Metadata, n_parameters=3>,
 <ParameterGroup name=Emission Lines, n_parameters=13>,
 <ParameterGroup name=Kinematics, n_parameters=6>,
 <ParameterGroup name=Spectral Indices, n_parameters=1>,
 <ParameterGroup name=NSA Catalog, n_parameters=11>]

Go ahead and try to create some new searches on your own from the parameter list. Please feel free to also try out the some of the same search on the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)**.

## Returning Bonus Parameters

Often you want to run a query and see the value of parameters that you didn't explicitly search on. For instance, you want to find galaxies above a redshift of 0.1 and would like to know their RA and DECs.

In Marvin-tools, this is as easy as specifying the **`returnparams`** option with either a string (for a single bonus parameter) or a list of strings (for multiple bonus parameters).

In [18]:
myquery5 = 'nsa.z > 0.1'
bonusparams5 = ['cube.ra', 'cube.dec']
# bonusparams5 = 'cube.ra'  # This works too

q5 = Query(search_filter=myquery5, returnparams=bonusparams5)
r5 = q5.run()

Your parsed filter is: 
nsa.z>0.1


In [19]:
r5.results

<ResultSet(set=1/1, index=0:110, count_in_set=110, total=110)>
[ResultRow(mangaid=u'1-25937', plate=7991, plateifu=u'7991-12705', ifu_name=u'12705', ra=261.115608768, dec=56.2960102323, z=0.113153),
 ResultRow(mangaid=u'1-25911', plate=7991, plateifu=u'7991-9102', ifu_name=u'9102', ra=260.346989853, dec=55.876526099, z=0.139784),
 ResultRow(mangaid=u'1-22948', plate=7992, plateifu=u'7992-9102', ifu_name=u'9102', ra=254.542083999, dec=62.4156479613, z=0.119399),
 ResultRow(mangaid=u'1-93471', plate=8483, plateifu=u'8483-6104', ifu_name=u'6104', ra=247.694974246, dec=47.7947656349, z=0.127959),
 ResultRow(mangaid=u'1-93872', plate=8555, plateifu=u'8555-6103', ifu_name=u'6103', ra=247.097142285, dec=43.9576126147, z=0.139743),
 ResultRow(mangaid=u'1-93876', plate=8484, plateifu=u'8484-9101', ifu_name=u'9101', ra=246.942944115, dec=44.177521384, z=0.139404),
 ResultRow(mangaid=u'1-93876', plate=8555, plateifu=u'8555-3704', ifu_name=u'3704', ra=246.942944115, dec=44.177521384, z=0.139404),


Next time, we'll take a closer look at the **`Results`** class and its built in MaNGA convenience functions.

## Useful Resources

Check out these pages on the Marvin Docs site for more **`Query`** tips and tricks.

- **[Query](https://api.sdss.org/doc/manga/marvin/refactor/query.html)**
- **[Example Queries](https://api.sdss.org/doc/manga/marvin/refactor/tutorials/query-examples.html)**
- **[Results](https://api.sdss.org/doc/manga/marvin/refactor/results.htmlpages)** 
- **[Boolean Search Tutorial](https://api.sdss.org/doc/manga/marvin/refactor/tutorials/boolean-search-tutorial.html)**